Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 3: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [30]:
class Literal:
    
    def __init__(self,name, val=1):
        '''Defining attributes of Literal objects'''
        self.name = name #literal name
        self.val = val #truth value of literal set to 1 unless there is - in front of it.
        if self.val == 1: #if value is 1, it is not inverted/negated
            self.inverted = False
        else: 
            self.inverted = True #if val!=1 then it is negated/inverted
            
        
    def __neg__(self):
        '''Creates and returns a new negated object of original literal.'''
        if not self.inverted:
            #only applies if the item is not already negated 
            return Literal(self.name,-self.val)
    
    
    def __str__(self):
        '''Returns a string representation of the literal object.'''
        if self.inverted:
            return '-'+self.name #if negated, put - in front of it
        return self.name #else, only print name as str

    
    def __repr__(self):
        '''Returns a printable version of the literal object. '''
        if self.inverted:
            return '-'+self.name
        return self.name
        
    
    def __eq__(self, other):
        '''Returns true if two literal objects are the same given name and sign.'''
        return self.name == other.name and self.inverted == other.inverted
    
    
    def __hash__(self):
        '''Returns hash value of a the literal object's name, to be hashed in lists and sets.'''
        return hash(self.name)



<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [6]:
from collections import Counter 

def puresymbol(kb):
    '''Returns a list of all pure symbols in a KB.
    Input: 
    kb: list of sets
    --
    Output:
    p: list of Literals 
    '''
    s = [] #list for Literal objects in kb
    sn = [] #list for Literal names (str) in kb
    #for loop to enumerate s and sn lists using Counter
    for i in kb: 
        for j in i:
            if j not in s:
                s.append(j)
                sn.append(j.name)   
    #if item is counted twice, it means it is not a pure symbol
    snc = Counter(sn)
    cc = [] #list for literal names of pure symbols
    #get literal names of pure  symbols
    for i, j in snc.items():
        if j == 1: 
            cc.append(i)
    p = [] #list for pure symbols as Literal objects
    for k in cc: 
        for l in s:
            if k == l.name:
                p.append(l)    
    return p 
    

def unitclause(kb):
    '''Returns a list of all unit clauses in a kb
    Input:
    kb: list of sets i.e. the knowledge base
    ---
    Output: 
    uc: list of sets where sets are unit clauses
    '''
    uc = []
    for m in kb:
        if len(m) == 1: #if length is 1, it is a unit clause
            uc.append(m)     
    return uc


def termination_check(kb):
    '''Checks if a kb is true by iteratively checking if all clauses are true in kb.
    Input:
    kb: list of sets
    ---
    Ouputs:
    Boolean value 
    '''
    c = 0
    for cls in kb: 
        #for clause in sentence
        for lit in cls:
            #for literal in clause
            if not lit.inverted:
                #if a literal is true, clause is true 
                c += 1
                break 
    if len(kb) == c: 
        #every clause is true. return True
        return True 
    else: 
        #if every clause is not True, KB is false
        return False 

    
def assign(kb,lit,val):
    '''Assigns True or False to all instances of a Literal (lit) in kb depending on val
    Inputs: 
    kb: list of sets
    lit: Literal object
    val: boolean
    determines the value to be assigned
    ---
    Outputs:
    the new kb with assigned values
    '''
    for g in kb:
        for h in g:
            #replace all instances of literal in KB with a literal of the assigned truth value. 
            if h.name == lit.name:
                if val: #if true, assign all corresponding literals to True truth val
                    g.remove(h)
                    g.add(lit)
                else:
                    g.remove(h)
                    g.add(-lit) #else, negate all literals
    return kb

def get_model(kb):
    '''Returns a model from a given KB'''
    model = {}
    for clause in kb:
        for literal in clause: 
            model[literal] = not literal.inverted 
    return model 

def DPLL(KB, symbols, model):
    '''Recursive algorithm to perform DPLL'''
    #check if given kb is satisfiable 
    if termination_check(kb):
        if not model:
            model = get_model(kb)
            continue
        return True, model 
    
    #base case -- 
    #this happens when there are no items in symbols to pop 
    #and satisfiability hasn't been reached either.
    #means KB is not satisfiable through backtracking assignment 
    if not symbols:
        return False, None 
    
    #recursive case -- pop items from symbols, assign to model, and check if satisfiable  
    else:
        literal = symbols.pop()
        #assign true and false to a literal and apply both to see if one works 
        kb1 = assign(KB,literal,True)
        kb2 = assign(KB,literal,False)
        return DPLL(kb1,symbols,True) or DPLL(kb2,symbols,model[literal]=False)
    
    #if the model gets to this point, a satisfiable model has been found.
    return True, model  
    

def DPLL_Satisfiable(KB, heuristic_level=0):
    ''' satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{B},{-A,C,D}]
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    #extract a list of all symbols in KB
    s = []
    s_names = []
    for cl in KB:
        for lt in cl:
            if lt not in s: 
                s.append(lt)
                s_names.append(lt.name)
    s_count = Counter(s_names) #use counter to enumerate symbols
    symbs = set() #set to save all symbols in KB
    for x,y in s_count.items(): 
        symbs.add(x)
      
    mod = {}
    #apply the DPLL recursive function to the symbol, model, and kb
    return DPLL(KB,symbs,mod)

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [27]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

# YOUR CODE HERE
KB = [{-A,B,E},{-B},{-E,A},{-E,D},{-C,-F,-B},{-E,B},{-B,F},{-B,C}]


<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [15]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [28]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

AttributeError: 'NoneType' object has no attribute 'name'

In [ ]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [ ]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [ ]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], ['B', 'A', 'C', 'F', 'E', 'A']])

symbol_trace

In [ ]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'], ['D', 'B', 'A', 'F', 'C', 'E', 'A'], ['D', 'B', 'A', 'C', 'F', 'E', 'A']])

In [ ]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'A', 'F', 'C', 'E', 'A'], ['D', 'B', 'A', 'C', 'F', 'E', 'A']])